In [ ]:
# Imports: Security, Networking, and Telemetry libraries
import paho.mqtt.client as mqtt
import ssl
import time
import random as rnd
import hashlib
import hmac
import json
import logging

# Logger setup
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Security Tokens
ACCESS_TOKEN1 = '6O4vviHAJG252MIUHKUn'
ACCESS_TOKEN2 = 'j2QZULfsKWRMIlVsO9Ni'

# Broker Configuration (TLS enabled)
broker = 'demo.thingsboard.io'
port = 8883  # Standard secure MQTT port
tls_ca_cert = '/etc/ssl/certs/ca-certificates.crt'  # Adjust to local system

# Helper: generate HMAC signature for payload integrity
def sign_payload(payload, key):
    return hmac.new(key.encode(), payload.encode(), hashlib.sha256).hexdigest()

# Telemetry data generators
def get_temperature():
    return '%.2f' % rnd.uniform(-50, 50) + ' Celsius'

def get_humidity():
    return '%.2f' % rnd.uniform(0, 100) + '%'

def get_wind_direction():
    return str(rnd.randrange(0, 360)) + ' degrees'

def get_wind_intensity():
    return str(rnd.randrange(0, 100)) + ' m/s'

def get_rain_height():
    return str(rnd.randrange(0, 50)) + ' mm/h'

def get_payload():
    payload_dict = {
        'Temperature': get_temperature(),
        'Humidity': get_humidity(),
        'Wind_direction': get_wind_direction(),
        'Wind_intensity': get_wind_intensity(),
        'Rain_height': get_rain_height()
    }
    payload_json = json.dumps(payload_dict)
    signature = sign_payload(payload_json, ACCESS_TOKEN1)
    return json.dumps({'data': payload_dict, 'signature': signature})

# MQTT client with layered security
def setup_client(client_id, access_token):
    client = mqtt.Client(client_id)
    client.username_pw_set(access_token)
    client.tls_set(ca_certs=tls_ca_cert, certfile=None, keyfile=None, tls_version=ssl.PROTOCOL_TLS_CLIENT)
    client.tls_insecure_set(False)  # Enforce proper TLS verification
    client.on_publish = lambda c, u, r: logging.info(f"Data published by {client_id}")
    client.connect(broker, port, keepalive=60)
    return client

client1 = setup_client('secure_control1', ACCESS_TOKEN1)
client2 = setup_client('secure_control2', ACCESS_TOKEN2)

# Layered Security: Segmentation & Monitoring
while True:
    try:
        payload1 = get_payload()
        payload2 = get_payload()

        # Rate limiting example (simple)
        time.sleep(1)

        ret1 = client1.publish('v1/devices/me/telemetry', payload1)
        ret2 = client2.publish('v1/devices/me/telemetry', payload2)

        logging.info(f'Payload1: {payload1}')
        logging.info(f'Payload2: {payload2}')
        time.sleep(5)
    except Exception as e:
        logging.error(f'Error in telemetry loop: {str(e)}')
